In [1]:
import numpy


In [57]:
# mesh paramters
nx = 81
dx = 0.25
nt = 6
dt = 0.002
gamma = 1.4

x = numpy.linspace(-10,10,nx)

In [106]:
# functions
# initial condition
def init_cond(nx):
    """
    Creates initial conditions for shock tube problem
    
    returns u which holds all initial conditions for u1, u2 and u3
    """
    # initialize rho
    rho_init = numpy.zeros(nx)
    rho_init[:int((nx-1)/2)] = 1. # left side density = 1 kg/m^3
    rho_init[int((nx-1)/2):] = 0.125 # right side density = 0.125 kg/m&3
    
    # initialize u
    u_init = numpy.zeros(nx)
    
    #initialize pressure
    p_init = numpy.zeros(nx)
    p_init[:int((nx-1)/2)] = 100e3 # left side pressure = 100k Pa
    p_init[int((nx-1)/2):] = 10e3  # right side pressure = 10k Pa
    
    # initialize u
    u = numpy.ndarray(shape=(3, nx))
    u[0,:] = rho_init       # puts rho_init into each rho to clear junk
    u[1,:] = rho_init*u_init# same for rho*u
    u[2,:] = rho_init*(p_init/((gamma-1)*rho_init) + u_init**2/2)
    return u

# flux
def computeF(u):
    u1 = u[0]
    u2 = u[1]
    u3 = u[2]
    return [u2,
            u2**2/2 + (gamma-1)*(u3 - 1/2*u2**2/u1),
            (u3 + (gamma - 1)*(u3 - 1/2*u2**2/u1))*u2/u1]

# Richtmyer method
def richtmyer(u, dx, dt, nt):
    
    # initialize our multidimensional results array with dimension 3 by nt by nx
    u_n = numpy.ndarray(shape=(3, nt, nx))
    for i in range(0,nt):
        u_n[:,i,:] = u[:,:].copy() # replaces junk with initial condition
    
    # intermediate time variable array:
    u_half = numpy.zeros(shape=(3,nx))
    u_half = u_n[:,0,:].copy()
    F_half = numpy.zeros(shape=(3,nx))
    
    for t in range(1,nt):
        # first step
        F_half[:,:] = computeF(u_n[:,t,:])
        u_half[:,:-1] = 0.5*(u_n[:,t,1:] + u_n[:,t,:-1]) - dt/(2*dx)*(F_half[:,1:] - F_half[:,:-1])
        # second step
        F = computeF(u_half)
        u_n[:,t,1:] = u[:,1:] - dt/dx*(F[1:] - F[:-1])
        u = u_n[:,t].copy()
    return u_n

In [107]:
# this block solves the problem
u = init_cond(nx)
u_n = richtmyer(u,dx,dt,nt)

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [55]:
# testing block
